# Merged codebase for the entire ML/Computer Vision parts of the project

###### *This file needs to be placed on ../Backend/*

### Imports packages needed and defines GLOBAL variables

In [1]:
# All imports needed
from ultralytics import YOLO
from PIL import Image
import csv
from utils import * # GLOBAL Parameters defined here

if not os.path.isdir(tmp_dir):
    os.makedirs(tmp_dir)

### Loads pre-trained models

In [2]:
cropResistor = YOLO("YOLO_Cropper\\latest\\best.pt")
findColorBands = YOLO("YOLO_Segmenter\\latest\\best_v10.pt")

### Code used for extracting and handling color classes

In [3]:
# Extracts the CSV files' color data to create color classes
def extract_colors(directory):
    classes = dict()
    for file in os.listdir(directory):
        if os.path.isfile(file):
            ext = file.split('.')[-1]
            name = ".".join(file.split('.')[:-1])
            if ext == "csv":
                with open(file) as csv_file:
                    csv_reader = csv.reader(csv_file, delimiter=',')
                    line_count = 0
                    summ = np.array([0,0,0])
                    for row in csv_reader:
                        H = int(row[0])
                        S = int(row[1])
                        V = int(row[2])
                        hsv = np.array([H,S,V])
                        line_count += 1
                        summ += hsv
                
                classes[name] = summ/line_count
    return classes

### Merged inference code

In [4]:
file = "Functionality Tests\\0009.png"

In [5]:
# Runs crop detection model on file
cropped_inf = cropResistor(file, conf=0.6)

# Crops and saves original file to a temporary location
if len(cropped_inf[0].boxes.data):
    cropped = cropImage(cv.imread(file), cropped_inf[0].boxes.cpu().data[0])
else:
    cropped = cv.imread(file)
Image.fromarray(cv.cvtColor(cropped, cv.COLOR_BGR2RGB)).save(tmp_crop)


image 1/1 e:\Users\jhony\Documents\Jhony\Universidade\UTFPR\2023.2\Oficinas\resistor-wizard\Backend\Functionality Tests\0009.png: 640x640 (no detections), 46.9ms
Speed: 133.7ms preprocess, 46.9ms inference, 499.8ms postprocess per image at shape (1, 3, 640, 640)


In [6]:
# Runs color bands segmentation model on cropped inference
if len(cropped_inf[0].boxes.data):
    colorbands_inf = findColorBands(tmp_crop, save=True, conf=0.6, iou=0.3)
else:
    colorbands_inf = findColorBands(file, save=True, conf=0.6, iou=0.3)


WARNING  NMS time limit 0.550s exceeded
image 1/1 e:\Users\jhony\Documents\Jhony\Universidade\UTFPR\2023.2\Oficinas\resistor-wizard\Backend\Functionality Tests\0009.png: 640x640 4 Color-Bands, 99.1ms
Speed: 11.6ms preprocess, 99.1ms inference, 1664.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to e:\Users\jhony\Documents\Jhony\Universidade\UTFPR\2023.2\Oficinas\resistor-wizard\runs\segment\predict


In [9]:
img = file
inference = colorbands_inf
data = 255

image = cv.imread(img)
masks = [Mask(image) for i in range(len(inference[0].masks.xy))]
"""
for i in range(len(inference[0].masks.xy)):
    polygon = inference[0].masks.xy[i].astype(int)
    masks[i].bbox = inference[0].boxes.cpu().data[i]
    masks[i].contour = polygon.reshape((-1, 1, 2))  # Reshape to match the format expected by cv.drawContours
    masks[i].mask = cv.fillPoly(masks[i].mask, [masks[i].contour], tuple([data]*3)) # converts polygon points to pixels inside it
    masks[i].sample_avg_color()
"""

'\nfor i in range(len(inference[0].masks.xy)):\n    polygon = inference[0].masks.xy[i].astype(int)\n    masks[i].bbox = inference[0].boxes.cpu().data[i]\n    masks[i].contour = polygon.reshape((-1, 1, 2))  # Reshape to match the format expected by cv.drawContours\n    masks[i].mask = cv.fillPoly(masks[i].mask, [masks[i].contour], tuple([data]*3)) # converts polygon points to pixels inside it\n    masks[i].sample_avg_color()\n'

In [18]:
i = 0
pol = inference[0].masks.xy[i].astype(int)
#bbox = inference[0].boxes.cpu().data[i]
contour = pol.reshape((-1, 1, 2)) 
mask = cv.fillPoly(masks[i].mask, [contour], tuple([data]*3)) 
pol

array([[365, 310],
       [363, 311],
       [360, 311],
       [358, 312],
       [357, 312],
       [356, 313],
       [356, 315],
       [355, 316],
       [355, 367],
       [356, 368],
       [356, 372],
       [357, 373],
       [357, 376],
       [358, 377],
       [358, 380],
       [360, 381],
       [360, 385],
       [361, 386],
       [361, 387],
       [362, 388],
       [362, 390],
       [363, 391],
       [363, 393],
       [365, 395],
       [365, 398],
       [366, 400],
       [366, 402],
       [367, 403],
       [367, 405],
       [368, 406],
       [368, 408],
       [370, 410],
       [370, 413],
       [371, 415],
       [371, 418],
       [372, 420],
       [372, 421],
       [373, 422],
       [373, 425],
       [375, 426],
       [375, 431],
       [376, 432],
       [376, 436],
       [377, 437],
       [377, 438],
       [378, 440],
       [378, 442],
       [380, 443],
       [380, 447],
       [381, 448],
       [381, 450],
       [382, 451],
       [382,

### Finds the order of bands

In [7]:
# Gets masks objects from the inference
masks, _ = timer(get_segmentation_masks, tmp_crop, colorbands_inf, printout=True)

# Sorts the masks by distance (it doesn't have a way to 
#    know in which direction to start, so by default it 
#    uses the leftmost (0,0) point as a beginning.
ordered = timer(order_masks, masks, colorbands_inf, printout=True)

Function  get_segmentation_masks  took 18.30s to run.
Function  order_masks  took 0.20s to run.


In [ ]:
len(ordered)

In [ ]:
for i in range(len(ordered)):
    print(f"BBOX: {ordered[i].bbox[:4]} \t\t HSV: {cvtBGR2HSV(ordered[i].avgColor[0], paint=True)}")

In [ ]:
# Retrieves the color name from masks average color
# Numbers on the side represent the number of samples used for each color's average
#  Files used to sample: res2.png, res.png, crop.jpg
COLOR_RANGES = {}
def sample_color_names(mask):
    pass